In [ ]:
from model.house_mask import HouseMask
from model.config import Config
from model.data_provider import DataProvider
from utils.anotations_parser import parse_anotation
from utils.plot_history import plot_history
from utils.showlist import show_image_list
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

import numpy as np
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow.keras.utils import plot_model

seed_value = 1234578790
seed(seed_value)
set_seed(seed_value)

config = Config()

In [ ]:
annotations = parse_anotation('data/train/annotations.json')
train_annotations = annotations#[0 : 550]
#test_annotations = annotations[550:]

# train_annotations = parse_anotation('data/train/annotations.json')
train_data_provider = DataProvider(config, train_annotations)

# test_annotations = parse_anotation('data/test/annotations.json')
test_data_provider = None # DataProvider(config, test_annotations)

In [ ]:
from model.unet_model import UNetModel, UNetModel2
# model_provider = UNetModel()
model_provider = UNetModel2()

In [ ]:
model = HouseMask(config, train_data_provider, test_data_provider, model_provider)

In [ ]:
model.model.summary()

In [ ]:
plot_model(model.model, show_shapes=True)

In [ ]:
model.train()

In [ ]:
plot_history(model.model.history)

In [ ]:
x,y = train_data_provider.__getitem__(0)
# x,y = test_data_provider.__getitem__(0)
predictions = model.predict(x)
predictions = np.array([model.prediction_to_mask(prediction) for prediction in predictions])

In [ ]:
image_count = len(x)
imges_lable = ['Original'] * image_count
anotation_lable = ['Anotation'] * image_count
prediction_lable = ['Prediction'] * image_count

titles = [None] * (image_count * 3)
titles[::3] = imges_lable
titles[1::3] = anotation_lable
titles[2::3] = prediction_lable

In [ ]:
detection_resutls = [None] * (image_count * 3)
detection_resutls[::3] = x
detection_resutls[1::3] = y
detection_resutls[2::3] = predictions

show_image_list(detection_resutls, titles, grid=False, num_cols=3, figsize=(20,150), title_fontsize=20)

In [ ]:
import glob
import cv2

def load_images():
    filenames = glob.glob("data/test/images/*")
    images = np.array([cv2.resize(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB), config.IMAGE_SIZE) for img_path in filenames])
    return images/255

In [ ]:
images = load_images()
predictions = model.predict(images)
predictions = np.array([model.prediction_to_mask(prediction) for prediction in predictions])

In [ ]:
image_count = len(images)
imges_lable = ['Original'] * image_count
prediction_lable = ['Prediction'] * image_count

titles = [None] * (image_count * 2)
titles[::2] = imges_lable
titles[1::2] = prediction_lable

detection_resutls = [None] * (image_count * 2)
detection_resutls[::2] = images
detection_resutls[1::2] = predictions

show_image_list(detection_resutls, titles, grid=False, num_cols=2, figsize=(20,50), title_fontsize=20)

In [ ]:
model.model.save('info/unet2_150_nval.h5')